In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict
%matplotlib widget

# Setup of a big dict.


In [2]:
n = 4

In [3]:
fibre_list = list()
#Generate all x,y,z fibres
for i in range(n):
    for j in range(n):
        tmp_fibre = list()
        tmp_fibre1 = list()
        tmp_fibre2 = list()
        for k in range(n):
            tmp_fibre.append( (i,j,k) ) 
            tmp_fibre1.append( (k,i,j) ) 
            tmp_fibre2.append( (j,k,i) ) 
        fibre_list.append(tmp_fibre)
        fibre_list.append(tmp_fibre1)
        fibre_list.append(tmp_fibre2)


#Generate diagonal fibres
for k in range(n):
    tmp_fibre = list()
    tmp_fibre1 = list()
    tmp_fibre2 = list()
    tmp_fibre3 = list()
    tmp_fibre4 = list()
    tmp_fibre5 = list()
    for i in range(n):
        #xz plane 
        tmp_fibre.append( (n-i-1,k,i) )
        tmp_fibre1.append( (i,k,i) )

        #xy plane
        tmp_fibre2.append( (i,n-i-1,k) )
        tmp_fibre3.append( (i,i,k) )

        #yz plane
        tmp_fibre4.append( (k,i,n-i-1) )
        tmp_fibre5.append( (k,i,i) )
    # print(tmp_fibre1)
    fibre_list.append(tmp_fibre)
    fibre_list.append(tmp_fibre1)
    fibre_list.append(tmp_fibre2)
    fibre_list.append(tmp_fibre3)
    fibre_list.append(tmp_fibre4)
    fibre_list.append(tmp_fibre5)

#Global diagonals
tmp_fibre = list()
tmp_fibre1 = list()
tmp_fibre2 = list()

for i in range(n):
    tmp_fibre.append( (i,i, i))
    tmp_fibre1.append( (n-1-i,i, i))
    tmp_fibre2.append( (i,n-1-i, i))

fibre_list.append(tmp_fibre)
fibre_list.append(tmp_fibre1)
fibre_list.append(tmp_fibre2)

print(len(fibre_list))
####

big_dict = dict()


for i in range(n):
    for j in range(n):
        for k in range(n):
            big_dict[f'{i}{j}{k}'] = [ fibre for fibre in fibre_list if (i,j,k) in fibre ]

# print(len(big_dict))

# fibre_rank = dict()
# for fibre in fibre_list:
#     #Player and rank
#     fibre_rank[f"{fibre}"] = [None,0]

75


# Environment

In [4]:

class Board():
    def __init__(self, n = 4, dimension = 3, big_dict =  big_dict, fibre_list = fibre_list):
        ''' 
        n = size of board (n x n x n) if dimension = 3.
        dimension = dimension of board
        
        Generate ALL possible fibres in a list where e.g. list[1] =  ( (0,0), (0,1), (0,2) ).
        Each fibre should only occur once in the list.
        Then create a dictionary with (x,y,z) as the index and a list of indices for fibres that intersect this point. also add a rank to each list
        );
        bigus_dict = { (0,0,0) : [0,2,7,65] }
        
        Next we only have to check these fibres for a win.
        
        Instead of checking all elements of each fibre, we create a another dict:
        
        fibre_rank = { 0: [0,--1] , ... }
        
        if the rank is -1, ignore this index and increase unwinnable fibres by 1. else increase each index of the fibres by 1. If a fibre has rank n we have a win.
        
        if unwinnable_fibres = total fibres, the game is a draw
        
        '''
        self.n = n
        self.dimension = dimension
        self.board = np.zeros([n,n,n])
        
        self.impossible_row_count = 0
        self.big_dict = big_dict
        
        self.fibre_rank = dict()
        for fibre in fibre_list:
            #Player and rank
            self.fibre_rank[f"{fibre}"] = [None,0]
        #This could be outsourced.
        
            
        # a = np.zeros([n,n])
        # for k in range(n):
        #     for i in range(n):
        #         for j in range(n):
        #             a[i,j] = len(big_dict[f'{i}{j}{k}'])
        #     print(k)
        #     print(a)
            
    def __repr__(self):
        for k in range(n):
            for j in range(n):
                print(f" "* (n -k),end='')
                print(self.board[k][j])
        return ''
    
    def reset(self):
        self.__init__()
        
    def board_status(self):
        print(f'Non zero elements on board: {np.count_nonzero(self.board)}')
    
    def board_vector(self):
        '''returns a vector with the board representation'''
        reshape = np.reshape(self.board, (-1,1))
        return reshape

    
    def addmove(self, player, coordinates ):
        '''
        add move and checks whether there is a win or a draw.
        '''
        x = coordinates[0]
        y = coordinates[1]
        z = coordinates[2]
        
        str_coords = str(x) + str(y) + str(z)
        
        #Check if valid move...
        
        self.board[n-z-1,n-y - 1,x] = player
        
        #Update all fibreranks that intersect with the given point
        for fibre in self.big_dict[str_coords]:
            #fibre is 1 fibre intersecting the given point.
            # print(fibre)
            
            tmp_rank = self.fibre_rank[f'{fibre}']
            
            if tmp_rank[1] == -1:
                pass
                
            # nobody has claimed this fibre
            elif not tmp_rank[0]:
                self.fibre_rank[f'{fibre}'] = [player, 1]
                
            # The fibre has been claimed by another asailant.
            elif tmp_rank[0] != player:
                # print("blocked!")
                self.fibre_rank[f'{fibre}'] = [None,-1]
                self.impossible_row_count += 1
                if self.impossible_row_count == self.n * self.n * 3 + 6*self.n + 3:
                    # print('DRAW')
                    return False
            # this is your fibre, increase its rank
            else:
                self.fibre_rank[f'{fibre}'][1] += 1
                if self.fibre_rank[f'{fibre}'][1] == 4:
                    #thou has won.
                    # print(f'WIN player {player}')
                    # print(f'win player {player}: {fibre} rank: { self.fibre_rank[f"{fibre}"][1] }')
                    return True
                
            
            # print(self.fibre_rank[f'{fibre}'] )
    
    def addmove_vector(self, player, vec_coords) -> None:
        ''' Takes in a vector as move input, this is usefull for the AI.
        It then processes this vector and treats it as a normal cartesian input.
        '''
        reshape = np.reshape(vec_coords, (self.n,self.n,self.n))
        
        #Get coordinates
        move = np.argwhere(reshape != 0)[0]
        
        #Convert coordinates to cartesian coordinates.
        tmp_x = move[0]
        move[0] = move[2]
        move[1] = self.n - move[1] - 1
        move[2] = self.n - tmp_x - 1
        return self.addmove(player,tuple( move))

    def impossible_moves(self):
        a = np.argwhere(self.board != 0)
        a0 = a[:,0].copy()
        a1 = a[:,1].copy()
        a2 = a[:,2].copy()
        b = np.zeros_like(a)

        b[:,0] = a2
        b[:,1] = n - a1 - 1
        b[:, 2] = n - a0 -1
        return tuple(map(tuple,b))
    
    def impossible_moves_vector(self):
        '''returns all impossible moves in vector representation'''
        return np.reshape(self.board != 0, (-1,1))
    
    def possible_moves(self):
        a = np.argwhere(self.board == 0)
        a0 = a[:,0].copy()
        a1 = a[:,1].copy()
        a2 = a[:,2].copy()
        b = np.zeros_like(a)

        b[:,0] = a2
        b[:,1] = n - a1 - 1
        b[:, 2] = n - a0 -1
        return tuple(map(tuple,b))
    




# Players

In [9]:
##  HERE we define the main player class. It's simple, only needing a name.
class Player:
    """Generate a tictactoe player.

    These operatioons are supported:
    * `choice`, to choose the next move based of an inputed Board.
    """

    def __init__(self,name:str):
        """Give player their name.
        """
        self.wincount = 0
        self.losscount =0 
        self.drawcount = 0
        self.name=name
                
    def __repr__(self) -> str:
        """Return player name. Computer representation.
        """
        return str(self.name)
    def add_win(self):
        self.wincount += 1
    def add_loss(self):
        self.losscount += 1
    def add_draw(self):
        self.drawcount += 1
    def resetcount(self):
        self.wincount = 0
        self.losscount = 0 
        self.drawcount = 0

# implement DL model
evolutionary dl. Give the model some parameters to adjust, instantiate a random instance, construct random 'gausian' balls around this instance (noise in parameters).
Make the different instances play 100 games against each other (somehow, make winners compete against winners).
After 1 round generate new instance center and repeat algorithm.

## Stucture of neural net
The neural net takes in a board position vector $x_0$ of size (n^3, 1), then multiplies this vector $x_0$ by weight matrix $w_0$ and ads a bias $b_0$, producing a new output vector $x_1$. We add some more of these layers.
Finaly the output shoud be a vector of size (n^3,1).

In [83]:
class neural_player(Player):
    def __init__(self,name:str, weights = None, biases = None):
        """Give player their name. Record callable function for choice
        """
        self.name = name
        self.wincount = 0
        self.losscount = 0 
        self.drawcount = 0
        
        self.layers = [n**3,128,48,n**3]
        
        if weights is None and biases is None:
            
            self.weight = []
            self.bias = []

            for index in range(len(self.layers) - 1) :

                self.weight.append(np.random.rand(self.layers[index + 1], self.layers[index]))

                self.bias.append( np.random.rand(self.layers[index + 1],1))
        else:
            self.weight = weights
            self.bias = biases
    def save_npy(self):
        np.save(f'{self.name}_weights', self.weight )
        np.save(f'{self.name}_biases', self.bias )

    def activation_function(self, x):
        return np.maximum(0,x)

    def choice(self,board:Board):
        prev_out= None
        for index in range(len(self.layers) - 1) :
            if prev_out is None:
                prev_out = board.board_vector()
                

            output = self.activation_function( np.matmul(self.weight[index], prev_out) + self.bias[index] )
            # print(f'Layer {index}, input= {np.shape(prev_out)} output = {np.shape(output)}')
            prev_out = output
            
        

        mask = board.impossible_moves_vector()
        output[mask] = 0 #only select the output nodes that are actually possible.
        
        
        a = np.zeros([n**3])
        a[np.argmax(output)] = 1 #Select the output node w the highest value.
        return a
    #This needs to get cleaned
    def add_win(self):
        self.wincount += 1
    def add_loss(self):
        self.losscount += 1
    def add_draw(self):
        self.drawcount += 1

class random_player(Player):
    def __init__(self,name:str):
        """Give player their name. Record callable function for choice
        """
        self.name = name
        self.wincount = 0
        self.losscount = 0 
        self.drawcount = 0
    def choice(self,board:Board):
        choice_coords = tuple(random.choice(board.possible_moves()))
        x = choice_coords[0]
        y= choice_coords[1]
        z = choice_coords[2]
        
        tmp = np.zeros( (board.n, board.n, board.n)) 
        tmp[n-z-1,n-y - 1,x] = 1
        return np.reshape(tmp, (-1,1))
    def add_win(self):
        self.wincount += 1
    def add_loss(self):
        self.losscount += 1
    def add_draw(self):
        self.drawcount += 1
    def resetcount(self):
        self.wincount = 0
        self.losscount = 0 
        self.drawcount = 0
    

# Play games

In [68]:
import time




def start_competition(player1, player2, iterations = 5000):
    board = Board()
    player_list = [player1, player2]
    matchcount = 0

    curr_index = 0
    uncurr_index = 1

    tic = time.perf_counter()
    player1.resetcount()
    player2.resetcount()
    while matchcount != iterations:

            # print(f"\rIter {asdf} ", end="")



            choice = player_list[curr_index].choice(board)
            result = board.addmove_vector(curr_index +1 ,choice)

            # result = board.addmove(curr_index +1 ,player_list[curr_index].choice(board))
            # print(f'Player: {curr_index + 1}, choice: {choice}', end='')
            # board.board_status()
            # print(board)

            if result is None: #no draw or win
                tmp =  curr_index
                curr_index = uncurr_index
                uncurr_index = tmp
            elif result: #win
                player_list[curr_index].add_win()
                player_list[uncurr_index].add_loss()
                matchcount +=1
                # print(board)
                board.reset()
                # print(f'MATCH {matchcount}')
                # print("WIN")


            else: #draw
                player_list[curr_index].add_draw()
                player_list[uncurr_index].add_draw()
                matchcount += 1
                board.reset()
                print("ELSE")
                # print(f'MATCH {matchcount}')

        # print(board) 
    print()
    print(f"{player1}, wincount = {player1.wincount}, losscount = {player1.losscount}, drawcount = {player1.drawcount}")
    print(f"{player2}, wincount = {player2.wincount}, losscount = {player2.losscount}, drawcount = {player2.drawcount}")
    toc = time.perf_counter()
    print(f"Took {toc - tic:0.4f} seconds")
    
    total_games = player1.wincount + player2.wincount
    
    if player1.wincount > player2.wincount:
        return [player1, player1.wincount / total_games]
    else:
        return [player2,player2.wincount / total_games]
    
    




## Find a random neural player that out performs the random player

In [63]:
randomPlayer =random_player("player_rand")
iterations = 100
result_dict = dict()
for i in range(iterations):
    playertmp = neural_player(f'Player_{i}')
        
    winning_player = start_competition(randomPlayer, playertmp,5000)
    if winning_player[0].name != 'player_rand':
        result_dict[i] = winning_player



player_rand, wincount = 2101, losscount = 2899, drawcount = 0
Player_0, wincount = 2899, losscount = 2101, drawcount = 0
Took 22.3322 seconds
ELSE

player_rand, wincount = 1365, losscount = 3634, drawcount = 1
Player_1, wincount = 3634, losscount = 1365, drawcount = 1
Took 21.1187 seconds

player_rand, wincount = 2154, losscount = 2846, drawcount = 0
Player_2, wincount = 2846, losscount = 2154, drawcount = 0
Took 21.6818 seconds

player_rand, wincount = 2396, losscount = 2604, drawcount = 0
Player_3, wincount = 2604, losscount = 2396, drawcount = 0
Took 22.8221 seconds

player_rand, wincount = 1950, losscount = 3050, drawcount = 0
Player_4, wincount = 3050, losscount = 1950, drawcount = 0
Took 22.1407 seconds

player_rand, wincount = 2314, losscount = 2686, drawcount = 0
Player_5, wincount = 2686, losscount = 2314, drawcount = 0
Took 21.4184 seconds

player_rand, wincount = 2008, losscount = 2992, drawcount = 0
Player_6, wincount = 2992, losscount = 2008, drawcount = 0
Took 21.8825 se

In [64]:
best_player = None
best_player_score = 0
best_player_index = None
for key,val in result_dict.items():
    if val[1] > best_player_score and val[0].name != 'player_rand':
        best_player = val[0]
        best_player_score = val[1]
        best_player_index = key
print(best_player, best_player_score)


Player_85 0.8672


In [65]:
randomPlayer =random_player("player_rand")

player_to_test = result_dict[best_player_index][0]
start_competition(player_to_test,randomPlayer, 100000)

ELSE
ELSE

Player_85, wincount = 86107, losscount = 13891, drawcount = 2
player_rand, wincount = 13891, losscount = 86107, drawcount = 2
Took 305.4262 seconds


[Player_85, 0.8610872217444349]

In [66]:
player_to_test.save_npy()

In [85]:

player1 = neural_player( "p1", np.load("Player_16_weights.npy", allow_pickle=True), np.load("Player_16_biases.npy", allow_pickle=True))
player2 = neural_player( "p2", np.load("Player_85_weights.npy", allow_pickle=True), np.load("Player_85_biases.npy", allow_pickle=True))